In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import datetime as dt
import pprint as pp
import pandas as pd
import math
from IPython.display import display
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import StandardScaler


In [15]:
df = pd.read_pickle('../data/player_game_table.pkl')
print(df.columns.values)


ave_features = ['ast','blk','min', 'pts','reb',
                  'stl','to','fantasy_points']
input_features = ['team_against','player_team', 'min_points', 'max_points',
                  'position', 'home_game', 'ave_points_team', 'ave_points_vs_team', 'name']

features = {}
for feature in ave_features:
    features[feature] = []
for feature in input_features:
    features[feature] = []
features['output_feature'] = []

['ast' 'blk' 'comment' 'dreb' 'fg3_pct' 'fg3a' 'fg3m' 'fg_pct' 'fga' 'fgm'
 'ft_pct' 'fta' 'ftm' 'game_date' 'game_id' 'gid' 'min' 'oreb' 'pf'
 'player_id' 'player_name' 'plus_minus' 'pts' 'reb' 'start_position' 'stl'
 'team_abbreviation' 'team_city' 'team_id' 'to' 'days_since_injury'
 'season' 'fantasy_points']


In [17]:
def calculate_team_stats(ind_game, model_features, past_season_games):
    game_box_stats = pd.read_pickle('../data/team_game_stats.pkl')
    overview = pd.read_pickle('../data/game.pkl')
    teams = pd.read_pickle('../data/team.pkl')
    
    team_id = float(ind_game.team_id)
    both_team_stats = game_box_stats.loc[game_box_stats.game_id == ind_game.game_id]
    vs_team_id = both_team_stats.loc[both_team_stats["team_id"] != team_id]
    
    vs_team_id = vs_team_id['team_id']
    vs_team_id = float(vs_team_id)
    game_ids = past_season_games.loc[past_season_games.team_id == str(team_id)].game_id.unique().tolist()
    vs_game_ids = past_season_games.loc[past_season_games.team_id == str(vs_team_id)].game_id.unique().tolist()
    
    team_id = int(team_id)
    vs_team_id = int(vs_team_id)
    
    team_abbrv_row = teams.loc[teams.team_id == str(vs_team_id)]
    team_abbrv = team_abbrv_row.team_abbreviation.values[0]
    model_features['team_against'].append(team_abbrv)
    model_features['player_team'].append(ind_game.team_abbreviation)
    model_features['home_game'].append(True if len(overview.loc[(overview.game_id == ind_game.game_id) & (overview.home_team_id == str(team_id))]) > 0 else False)

    
    pts = game_box_stats.loc[
        (game_box_stats.game_id.isin(game_ids)) & (game_box_stats.team_id == team_id)].pts.tolist()
    vs_pts = game_box_stats.loc[
        (game_box_stats.game_id.isin(vs_game_ids)) & (game_box_stats.team_id == vs_team_id)].pts.tolist()
    
    if (len(pts) > 0):
         model_features['ave_points_team'].append(np.average(pts))
    else:
        model_features['ave_points_team'].append(0)
        
    if (len(vs_pts) > 0):
         model_features['ave_points_vs_team'].append(np.average(vs_pts))
    else:
        model_features['ave_points_vs_team'].append(0)
    
    

In [18]:
def create_nfl_features(df, feats, model_features):
    
   
    rows = df
    rows = rows.sort_values("game_date")
    rows = rows.loc[rows.season.isin(['2017-2018','2016-2017','2015-16'])]
    count = 0
    for i in range(1, len(rows)-1):
        if (not count%100):
            print(count, " out of ", len(rows))
        count += 1
        average_rows = rows.loc[(rows.game_date<rows.iloc[i].game_date) 
                                & (rows.season == rows.iloc[i].season) 
                                & (rows.player_id == rows.iloc[i].player_id)]
        #X[(i - n),0] = pid
        
        
        
        if len(average_rows) > 0:
            
            #averages
            for a,feat in enumerate(feats):
                features[feat].append(np.average(average_rows[feat].tolist()))

            
            calculate_team_stats(rows.iloc[i], model_features, df.loc[(df.game_date<df.iloc[i].game_date) 
                                & (df.season == df.iloc[i].season)])
            model_features["max_points"].append(max(average_rows["fantasy_points"].tolist()))
            model_features["min_points"].append(min(average_rows["fantasy_points"].tolist()))
            model_features["name"].append(rows.iloc[i]["player_name"])
            model_features["position"].append(rows.iloc[i]["start_position"])
            model_features["output_feature"].append(rows.iloc[i]["fantasy_points"])
            
    #max/min
    return model_features

In [19]:
model_features = create_nfl_features(df, ave_features, features)
print(model_features)

with open('nfl_featuers.pkl', 'wb') as fp:
    pickle.dump(features, fp, protocol=pickle.HIGHEST_PROTOCOL)


0  out of  52242
100  out of  52242
200  out of  52242
300  out of  52242
400  out of  52242
500  out of  52242
600  out of  52242
700  out of  52242
800  out of  52242
900  out of  52242
1000  out of  52242
1100  out of  52242
1200  out of  52242
1300  out of  52242
1400  out of  52242
1500  out of  52242
1600  out of  52242
1700  out of  52242
1800  out of  52242
1900  out of  52242
2000  out of  52242
2100  out of  52242
2200  out of  52242
2300  out of  52242
2400  out of  52242
2500  out of  52242
2600  out of  52242
2700  out of  52242
2800  out of  52242
2900  out of  52242
3000  out of  52242
3100  out of  52242
3200  out of  52242
3300  out of  52242
3400  out of  52242
3500  out of  52242
3600  out of  52242
3700  out of  52242
3800  out of  52242
3900  out of  52242
4000  out of  52242
4100  out of  52242
4200  out of  52242
4300  out of  52242
4400  out of  52242
4500  out of  52242
4600  out of  52242
4700  out of  52242
4800  out of  52242
4900  out of  52242
5000  out of

39600  out of  52242
39700  out of  52242
39800  out of  52242
39900  out of  52242
40000  out of  52242
40100  out of  52242
40200  out of  52242
40300  out of  52242
40400  out of  52242
40500  out of  52242
40600  out of  52242
40700  out of  52242
40800  out of  52242
40900  out of  52242
41000  out of  52242
41100  out of  52242
41200  out of  52242
41300  out of  52242
41400  out of  52242
41500  out of  52242
41600  out of  52242
41700  out of  52242
41800  out of  52242
41900  out of  52242
42000  out of  52242
42100  out of  52242
42200  out of  52242
42300  out of  52242
42400  out of  52242
42500  out of  52242
42600  out of  52242
42700  out of  52242
42800  out of  52242
42900  out of  52242
43000  out of  52242
43100  out of  52242
43200  out of  52242
43300  out of  52242
43400  out of  52242
43500  out of  52242
43600  out of  52242
43700  out of  52242
43800  out of  52242
43900  out of  52242
44000  out of  52242
44100  out of  52242
44200  out of  52242
44300  out of

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X,y)
#X = X[:-1]
#y = y[:-1]
X = X[1:]
y = y[1:]
print(np.shape(X))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=10,criterion = 'mae',n_jobs =8,max_features=10)
train_split = 3/4
train_n = int(len(y)*train_split)
regr.fit(X[:train_n],y[:train_n])

plt.plot(list(range(train_n,len(y))),regr.predict(X[train_n:]),'r')
plt.axhline(y = np.average(y))
plt.plot(list(range(train_n,len(y))),y[train_n:],'b')
plt.show()
print(regr.score(X[train_n:],y[train_n:]))